In [ ]:
pip install torch torchvision pillow tqdm pandas numpy


In [ ]:
import zipfile
import os

# Path to your uploaded ZIP file (update if your filename is different)
zip_path = "/content/student_resource.zip"  # or change this to match your uploaded filename

# Destination folder (where files will be extracted)
extract_to = "/content/amazon_ml_challenge"

# Create folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files extracted successfully to:", extract_to)

# List extracted contents
for root, dirs, files in os.walk(extract_to):
    level = root.replace(extract_to, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


Files extracted successfully to: /content/amazon_ml_challenge
amazon_ml_challenge/
  student_resource/
    .DS_Store
    Documentation_template.md
    sample_code.py
    README.md
    dataset/
      sample_test.csv
      train.csv
      sample_test_out.csv
      test.csv
    src/
      utils.py
      example.ipynb
      __pycache__/
        utils.cpython-37.pyc


In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import requests
from io import BytesIO

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms


In [ ]:
DATASET_DIR = "/content/amazon_ml_challenge/student_resource/dataset"
OUT_DIR = "/content/amazon_ml_challenge/artifacts_10k"
IMG_TRAIN_DIR = os.path.join(OUT_DIR, "images_train_10k")
IMG_TEST_DIR = os.path.join(OUT_DIR, "images_test_10k")
BATCH_SIZE = 32
N_RECORDS = 10000

os.makedirs(IMG_TRAIN_DIR, exist_ok=True)
os.makedirs(IMG_TEST_DIR, exist_ok=True)

In [ ]:
# STEP 1: LOAD DATA
train = pd.read_csv(os.path.join(DATASET_DIR, "train.csv")).head(N_RECORDS)
test = pd.read_csv(os.path.join(DATASET_DIR, "test.csv")).head(N_RECORDS)

In [ ]:
# STEP 2: IMAGE DOWNLOAD FUNCTION
def download_image(url, save_path):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content)).convert("RGB")
            img.save(save_path)
            return True
    except Exception:
        pass
    return False

# Download images
def download_images(df, out_dir):
    for _, row in tqdm(df.iterrows(), total=len(df)):
        img_path = os.path.join(out_dir, f"{row['sample_id']}.jpg")
        if not os.path.exists(img_path):
            success = download_image(row['image_link'], img_path)
            if not success:
                # Save a blank image placeholder
                Image.new('RGB', (224, 224), color=(255,255,255)).save(img_path)

print("Downloading training images...")
download_images(train, IMG_TRAIN_DIR)

print("Downloading test images...")
download_images(test, IMG_TEST_DIR)

100%|██████████| 10000/10000 [15:02<00:00, 11.08it/s]


100%|██████████| 10000/10000 [15:16<00:00, 10.91it/s]


In [ ]:
# STEP 3: DEFINE MODEL (ResNet50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model = nn.Sequential(*list(model.children())[:-1])  # remove classification layer
model.to(device)
model.eval()

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 198MB/s]


In [ ]:
#STEP 4: FEATURE EXTRACTION
def extract_features(img_dir, df):
    features = []
    img_paths = [os.path.join(img_dir, f"{sid}.jpg") for sid in df['sample_id']]

    with torch.no_grad():
        for i in tqdm(range(0, len(img_paths), BATCH_SIZE)):
            batch_paths = img_paths[i:i+BATCH_SIZE]
            imgs = []
            for path in batch_paths:
                try:
                    img = Image.open(path).convert("RGB")
                    imgs.append(transform(img))
                except Exception:
                    imgs.append(torch.zeros(3, 224, 224))

            imgs = torch.stack(imgs).to(device)
            outputs = model(imgs).squeeze(-1).squeeze(-1)  # shape (B, 2048)
            features.append(outputs.cpu().numpy())

    features = np.concatenate(features, axis=0)
    return features

print("Extracting train image embeddings...")
train_features = extract_features(IMG_TRAIN_DIR, train)
np.save(os.path.join(OUT_DIR, "image_embeddings_train.npy"), train_features)

print("Extracting test image embeddings...")
test_features = extract_features(IMG_TEST_DIR, test)
np.save(os.path.join(OUT_DIR, "image_embeddings_test.npy"), test_features)

print("Image feature extraction complete!")
print("Train shape:", train_features.shape, "| Test shape:", test_features.shape)

Extracting train image embeddings...


100%|██████████| 313/313 [05:53<00:00,  1.13s/it]


Extracting test image embeddings...


100%|██████████| 313/313 [05:53<00:00,  1.13s/it]

Image feature extraction complete!
Train shape: (10000, 2048) | Test shape: (10000, 2048)


In [ ]:
import os
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt

# CONFIG — change paths if needed
OUT_DIR = "/content/amazon_ml_challenge/artifacts_10k"
IMG_TRAIN_DIR = os.path.join(OUT_DIR, "images_train_10k")
IMG_TEST_DIR = os.path.join(OUT_DIR, "images_test_10k")

train_embed_path = os.path.join(OUT_DIR, "image_embeddings_train.npy")
test_embed_path = os.path.join(OUT_DIR, "image_embeddings_test.npy")

# 1 Check existence and shapes

if os.path.exists(train_embed_path) and os.path.exists(test_embed_path):
    train_embeds = np.load(train_embed_path)
    test_embeds = np.load(test_embed_path)
    print("Image embeddings found!")
    print("Train embeddings shape:", train_embeds.shape)
    print("Test embeddings shape:", test_embeds.shape)
else:
    print("Embedding files not found. Please re-run image_processing_10k.py first.")

# 2 Visualize a few sample images
def show_random_images(img_dir, n=5):
    img_files = os.listdir(img_dir)
    sample_files = random.sample(img_files, min(n, len(img_files)))
    plt.figure(figsize=(15, 3))
    for i, fname in enumerate(sample_files):
        img_path = os.path.join(img_dir, fname)
        img = Image.open(img_path)
        plt.subplot(1, n, i + 1)
        plt.imshow(img)
        plt.axis("off")
        plt.title(fname.split(".")[0])
    plt.show()

print("\nSample training images:")
show_random_images(IMG_TRAIN_DIR, n=5)

print("\nSample test images:")
show_random_images(IMG_TEST_DIR, n=5)

# 3️ Show some sample embeddings
if 'train_embeds' in locals():
    print("\nSample training embedding (first row):")
    print(train_embeds[0][:10], "...")  # first 10 values
    print("\nEmbedding stats:")
    print("Mean:", train_embeds.mean(), "Std:", train_embeds.std())


Image embeddings found!
Train embeddings shape: (10000, 2048)
Test embeddings shape: (10000, 2048)

Sample training images:



Sample test images:



Sample training embedding (first row):
[0.85714126 1.2193997  0.02255034 0.00204107 0.23258357 1.3792675
 0.28593892 0.03599072 0.16433601 0.09753118] ...

Embedding stats:
Mean: 0.43291974 Std: 0.44383723


In [ ]:
import shutil

# Folder you want to download
folder_path = "/content/amazon_ml_challenge"
output_filename = "/content/amazon_ml_challenge.zip"

# Create ZIP
shutil.make_archive("/content/amazon_ml_challenge", 'zip', folder_path)
print("Folder zipped!")

# Download ZIP
from google.colab import files
files.download(output_filename)


Folder zipped!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>